In [16]:
###################################
## comment
###################################
# few things to check -
#         1. data generation => n, d, s0, graph_type, sem_type = 200, 10, 40, 'ER', 'mlp'
#         2. env generation =>  noise_scales = [0.2, 1, 2, 5, 10]
#         3. nhidden=10, lambda1=0.01, lambda2=0.01, w_threshold=0.3, std_lambda=1/10/100/1000, learning_rate=1e-3/1e-2, max_iter=1000
#         4. in utils.count_accuracy() => is_dag() is disabled/enabled?

In [17]:
###################################
## install and import
###################################

In [18]:
# !pip install python-igraph
from notears.locally_connected import LocallyConnected
from notears.lbfgsb_scipy import LBFGSBScipy
from notears.trace_expm import trace_expm
import torch
import torch.nn as nn
import numpy as np
import math
import notears.utils as ut
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize

In [19]:
###################################
## class
###################################

In [20]:
class NotearsMLP(nn.Module):
    def __init__(self, dims, bias=True):
        super(NotearsMLP, self).__init__()
        assert len(dims) >= 2
        assert dims[-1] == 1
        d = dims[0]
        m1 = dims[1]        
        self.dims = dims
        # fc1: variable splitting for l1
        ##
        mask = torch.ones(d, m1, d)
        for i in range(d):
            for j in range(m1):
                for k in range(d):
                    if i==k:
                        mask[i, j, k] = 0
        mask = mask.view(d*m1, d)
        self.register_buffer('diag_mask', mask)
        ##
        self.fc1_pos = nn.Linear(d, d * dims[1], bias=bias)
        self.fc1_neg = nn.Linear(d, d * dims[1], bias=bias)
        self.fc1_pos.weight.bounds = self._bounds()
        self.fc1_neg.weight.bounds = self._bounds()
        # fc2: local linear layers
        layers = []
        for l in range(len(dims) - 2):
            layers.append(LocallyConnected(d, dims[l + 1], dims[l + 2], bias=bias))
        self.fc2 = nn.ModuleList(layers)

    def _bounds(self):
        d = self.dims[0]
        bounds = []
        for j in range(d):
            for m in range(self.dims[1]):
                for i in range(d):
                    if i == j:
                        bound = (0, 0)
                    else:
                        bound = (0, None)
                    bounds.append(bound)
        return bounds

    def forward(self, x):  # [n, d] -> [n, d]
        with torch.no_grad():
            self.fc1_pos.weight *= self.diag_mask
            self.fc1_neg.weight *= self.diag_mask
            
        x = self.fc1_pos(x) - self.fc1_neg(x)  # [n, d * m1]
        x = x.view(-1, self.dims[0], self.dims[1])  # [n, d, m1]
        for fc in self.fc2:
            x = torch.sigmoid(x)  # [n, d, m1]
            x = fc(x)  # [n, d, m2]
        x = x.squeeze(dim=2)  # [n, d]
        return x

    def h_func(self):
        """Constrain 2-norm-squared of fc1 weights along m1 dim to be a DAG"""
        d = self.dims[0]
        fc1_weight = self.fc1_pos.weight - self.fc1_neg.weight  # [j * m1, i]
        fc1_weight = fc1_weight.view(d, -1, d)  # [j, m1, i]
        A = torch.sum(fc1_weight * fc1_weight, dim=1).t()  # [i, j]
        h = trace_expm(A) - d  # (Zheng et al. 2018)
        # A different formulation, slightly faster at the cost of numerical stability
        # M = torch.eye(d) + A / d  # (Yu et al. 2019)
        # E = torch.matrix_power(M, d - 1)
        # h = (E.t() * M).sum() - d
        return h

    def l2_reg(self):
        """Take 2-norm-squared of all parameters"""
        reg = 0.
        fc1_weight = self.fc1_pos.weight - self.fc1_neg.weight  # [j * m1, i]
        reg += torch.sum(fc1_weight ** 2)
        for fc in self.fc2:
            reg += torch.sum(fc.weight ** 2)
        return reg

    def fc1_l1_reg(self):
        """Take l1 norm of fc1 weight"""
        reg = torch.sum(self.fc1_pos.weight + self.fc1_neg.weight)
        return reg

    @torch.no_grad()
    def fc1_to_adj(self) -> np.ndarray:  # [j * m1, i] -> [i, j]
        """Get W from fc1 weights, take 2-norm over m1 dim"""
        d = self.dims[0]
        fc1_weight = self.fc1_pos.weight - self.fc1_neg.weight  # [j * m1, i]
        fc1_weight = fc1_weight.view(d, -1, d)  # [j, m1, i]
        A = torch.sum(fc1_weight * fc1_weight, dim=1).t()  # [i, j]
        W = torch.sqrt(A)  # [i, j]
        W = W.cpu().detach().numpy()  # [i, j]
        return W

class NotearsSobolev(nn.Module):
    def __init__(self, d, k):
        """d: num variables k: num expansion of each variable"""
        super(NotearsSobolev, self).__init__()
        self.d, self.k = d, k
        self.fc1_pos = nn.Linear(d * k, d, bias=False)  # ik -> j
        self.fc1_neg = nn.Linear(d * k, d, bias=False)
        self.fc1_pos.weight.bounds = self._bounds()
        self.fc1_neg.weight.bounds = self._bounds()
        nn.init.zeros_(self.fc1_pos.weight)
        nn.init.zeros_(self.fc1_neg.weight)
        self.l2_reg_store = None

    def _bounds(self):
        # weight shape [j, ik]
        bounds = []
        for j in range(self.d):
            for i in range(self.d):
                for _ in range(self.k):
                    if i == j:
                        bound = (0, 0)
                    else:
                        bound = (0, None)
                    bounds.append(bound)
        return bounds

    def sobolev_basis(self, x):  # [n, d] -> [n, dk]
        seq = []
        for kk in range(self.k):
            mu = 2.0 / (2 * kk + 1) / math.pi  # sobolev basis
            psi = mu * torch.sin(x / mu)
            seq.append(psi)  # [n, d] * k
        bases = torch.stack(seq, dim=2)  # [n, d, k]
        bases = bases.view(-1, self.d * self.k)  # [n, dk]
        return bases

    def forward(self, x):  # [n, d] -> [n, d]
        bases = self.sobolev_basis(x)  # [n, dk]
        x = self.fc1_pos(bases) - self.fc1_neg(bases)  # [n, d]
        self.l2_reg_store = torch.sum(x ** 2) / x.shape[0]
        return x

    def h_func(self):
        fc1_weight = self.fc1_pos.weight - self.fc1_neg.weight  # [j, ik]
        fc1_weight = fc1_weight.view(self.d, self.d, self.k)  # [j, i, k]
        A = torch.sum(fc1_weight * fc1_weight, dim=2).t()  # [i, j]
        h = trace_expm(A) - d  # (Zheng et al. 2018)
        # A different formulation, slightly faster at the cost of numerical stability
        # M = torch.eye(self.d) + A / self.d  # (Yu et al. 2019)
        # E = torch.matrix_power(M, self.d - 1)
        # h = (E.t() * M).sum() - self.d
        return h

    def l2_reg(self):
        reg = self.l2_reg_store
        return reg

    def fc1_l1_reg(self):
        reg = torch.sum(self.fc1_pos.weight + self.fc1_neg.weight)
        return reg

    @torch.no_grad()
    def fc1_to_adj(self) -> np.ndarray:
        fc1_weight = self.fc1_pos.weight - self.fc1_neg.weight  # [j, ik]
        fc1_weight = fc1_weight.view(self.d, self.d, self.k)  # [j, i, k]
        A = torch.sum(fc1_weight * fc1_weight, dim=2).t()  # [i, j]
        W = torch.sqrt(A)  # [i, j]
        W = W.cpu().detach().numpy()  # [i, j]
        return W

In [21]:
###################################
## function
###################################

In [22]:
def squared_loss(output, target):
    n = target.shape[0]
    loss = 0.5 / n * torch.sum((output - target) ** 2)
    return loss

def dual_ascent_step(model, X, lambda1, lambda2, rho, alpha, h, rho_max):
    """Perform one step of dual ascent in augmented Lagrangian."""
    h_new = None
    optimizer = LBFGSBScipy(model.parameters())
    X_torch = torch.from_numpy(X)
    while rho < rho_max:
        def closure():
            optimizer.zero_grad()
            X_hat = model(X_torch)
            loss = squared_loss(X_hat, X_torch)
            h_val = model.h_func()
            penalty = 0.5 * rho * h_val * h_val + alpha * h_val
            l2_reg = 0.5 * lambda2 * model.l2_reg()
            l1_reg = lambda1 * model.fc1_l1_reg()
            primal_obj = loss + penalty + l2_reg + l1_reg
            # primal_obj = loss + penalty
            primal_obj.backward()
            return primal_obj
        optimizer.step(closure)  # NOTE: updates model in-place
        with torch.no_grad():
            h_new = model.h_func().item()
        if h_new > 0.25 * h:
            rho *= 10
        else:
            break
    alpha += rho * h_new
    return rho, alpha, h_new

def notears_nonlinear(model: nn.Module,
                      X: np.ndarray,
                      lambda1: float = 0.,
                      lambda2: float = 0.,
                      max_iter: int = 100,
                      h_tol: float = 1e-8,
                      rho_max: float = 1e+16,
                      w_threshold: float = 0.3*0.3):
    rho, alpha, h = 1.0, 0.0, np.inf
    for _ in range(max_iter):
        rho, alpha, h = dual_ascent_step(model, X, lambda1, lambda2,
                                         rho, alpha, h, rho_max)
        if h <= h_tol or rho >= rho_max:
            break
    W_est = model.fc1_to_adj()
    W_est[np.abs(W_est) < w_threshold] = 0
    return W_est

def notears_nonlinear_with_loss_std(model: nn.Module,
                                    X_list: list,  
                                    lambda1: float = 0.0,
                                    lambda2: float = 0.0,
                                    max_iter: int = 1000,
                                    h_tol: float = 1e-8,
                                    rho_max: float = 1e+16,
                                    w_threshold: float = 0.3*2.8,
                                    std_lambda: float = 1.0,
                                    lr: float = 1e-3):  

    for _ in range(max_iter):
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)  
        X_tensors = [torch.from_numpy(X).float().to(torch.double) for X in X_list]
        total_loss = 0
        individual_losses = []
        feature_wise_errors = []        
        for X in X_tensors:
            X_hat = model(X)
            loss = squared_loss(X_hat, X)
            individual_losses.append(loss)
            total_loss += loss 
            feature_errors = 0.5 / X.shape[0] * torch.sum((X_hat - X) ** 2, dim=0)
            feature_wise_errors.append(feature_errors)
        total_loss = total_loss / len(individual_losses)
        feature_wise_errors = torch.stack(feature_wise_errors) 
        feature_std_devs = torch.std(feature_wise_errors, dim=0)
        # print(total_loss, torch.mean(feature_std_devs))
        total_loss += std_lambda * torch.mean(feature_std_devs)
        l2_reg = 0.5 * lambda2 * model.l2_reg()
        l1_reg = lambda1 * model.fc1_l1_reg()
        total_loss += l2_reg + l1_reg
        # print(l2_reg, l1_reg)
        # print('---')
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
    W_est = model.fc1_to_adj()
    W_est[np.abs(W_est) < w_threshold] = 0
    return W_est


In [23]:
###################################
## initialize
###################################

In [24]:
torch.set_default_dtype(torch.double)
np.set_printoptions(precision=3)
ut.set_random_seed(123)
n, d, s0, graph_type, sem_type = 200, 10, 40, 'ER', 'mlp'
B_true = ut.simulate_dag(d, s0, graph_type)
np.savetxt('inputs/W_true.csv', B_true, delimiter=',')
# noise_scale = np.ones(d)
# X = ut.simulate_nonlinear_sem(B_true, n, sem_type, noise_scale)
# np.savetxt('X.csv', X, delimiter=',')
noise_scales = [0.2, 1, 2, 5, 10]
for i, noise_scale_value in enumerate(noise_scales):
    noise_scale = np.full(d, noise_scale_value)  
    X = ut.simulate_nonlinear_sem(B_true, n, sem_type, noise_scale)
    np.savetxt(f'inputs/X_{i}.csv', X, delimiter=',')  
    
X_0 = np.loadtxt('inputs/X_0.csv', delimiter=',')
X_1 = np.loadtxt('inputs/X_1.csv', delimiter=',')
X_2 = np.loadtxt('inputs/X_2.csv', delimiter=',')
X_3 = np.loadtxt('inputs/X_3.csv', delimiter=',')
X_4 = np.loadtxt('inputs/X_4.csv', delimiter=',')
X_list = [X_0, X_1, X_2, X_3, X_4]  # List of datasets
scaler = StandardScaler()
X_list_standardized = [scaler.fit_transform(X) for X in X_list]
X_combined = np.vstack([X for X in X_list_standardized]) 
print(X_combined.shape)
noise_scale_value_test = np.random.uniform(min(noise_scales), max(noise_scales))
noise_scale_test = np.full(d, noise_scale_value_test)
X_test = ut.simulate_nonlinear_sem(B_true, n, sem_type, noise_scale_test)
print(X_test.shape)

(1000, 10)
(200, 10)


In [25]:
###################################
## NOTEARS
###################################

In [26]:
list_fdr, list_shd, list_tpr, list_nnz = [], [], [], []
for i in range(5):
    model = NotearsMLP(dims=[d, 10, 1], bias=True)
    W_est = notears_nonlinear(model, X_combined, lambda1=0.01, lambda2=0.01)
    # assert ut.is_dag(W_est)
    np.savetxt('outputs/W_est.csv', W_est, delimiter=',')
    acc = ut.count_accuracy(B_true, W_est != 0)
    print(i, acc)
    list_fdr.append(acc['fdr'])
    list_shd.append(acc['shd'])
    list_tpr.append(acc['tpr'])
    list_nnz.append(acc['nnz'])    
print()
print()
print(f'FDR: {np.mean(list_fdr):.4f} ± {np.std(list_fdr):.4f}')
print(f'SHD: {np.mean(list_shd):.4f} ± {np.std(list_shd):.4f}')
print(f'TPR: {np.mean(list_tpr):.4f} ± {np.std(list_tpr):.4f}')
print(f'NNZ: {np.mean(list_nnz):.4f} ± {np.std(list_nnz):.4f}')
X_hat_test = model(torch.from_numpy(X_test).float().to(torch.double))
loss_test = squared_loss(X_hat_test, torch.from_numpy(X_test).float().to(torch.double))
print(f'Test squared loss on original values: {loss_test.item():.4f}')

0 {'fdr': 0.36666666666666664, 'tpr': 0.475, 'fpr': 2.2, 'shd': 24, 'nnz': 30}
1 {'fdr': 0.36666666666666664, 'tpr': 0.475, 'fpr': 2.2, 'shd': 24, 'nnz': 30}
2 {'fdr': 0.36666666666666664, 'tpr': 0.475, 'fpr': 2.2, 'shd': 24, 'nnz': 30}
3 {'fdr': 0.36666666666666664, 'tpr': 0.475, 'fpr': 2.2, 'shd': 24, 'nnz': 30}
4 {'fdr': 0.36666666666666664, 'tpr': 0.475, 'fpr': 2.2, 'shd': 24, 'nnz': 30}


FDR: 0.3667 ± 0.0000
SHD: 24.0000 ± 0.0000
TPR: 0.4750 ± 0.0000
NNZ: 30.0000 ± 0.0000
Test squared loss on original values: 725.9890


In [27]:
###################################
## invariant NOTEARS
###################################

In [28]:
lsl = [10, 100, 1000]
llr = [1e-3, 1e-2, 1e-1]

for sl in lsl:
    for lr in llr:
        print()
        print()
        print(f'std_lambda: {sl} and learning_rate: {lr}')
        list_fdr, list_shd, list_tpr, list_nnz = [], [], [], []
        for i in range(5):
            model = NotearsMLP(dims=[d, 10, 1], bias=True)
            W_est = notears_nonlinear_with_loss_std(model, X_list_standardized, lambda1=0.01, lambda2=0.01, std_lambda=sl, lr=lr)
            # assert ut.is_dag(W_est)
            np.savetxt('outputs/W_est2.csv', W_est, delimiter=',')
            acc = ut.count_accuracy(B_true, W_est != 0)
            print(i, acc)
            list_fdr.append(acc['fdr'])
            list_shd.append(acc['shd'])
            list_tpr.append(acc['tpr'])
            list_nnz.append(acc['nnz'])        
        print()
        print()
        print(f'FDR: {np.mean(list_fdr):.4f} ± {np.std(list_fdr):.4f}')
        print(f'SHD: {np.mean(list_shd):.4f} ± {np.std(list_shd):.4f}')
        print(f'TPR: {np.mean(list_tpr):.4f} ± {np.std(list_tpr):.4f}')
        print(f'NNZ: {np.mean(list_nnz):.4f} ± {np.std(list_nnz):.4f}')
        X_hat_test = model(torch.from_numpy(X_test).float().to(torch.double))
        loss_test = squared_loss(X_hat_test, torch.from_numpy(X_test).float().to(torch.double))
        print(f'Test squared loss on original values: {loss_test.item():.4f}')



std_lambda: 10 and learning_rate: 0.001
0 {'fdr': 0.6551724137931034, 'tpr': 0.25, 'fpr': 3.8, 'shd': 36, 'nnz': 29}
1 {'fdr': 0.5263157894736842, 'tpr': 0.225, 'fpr': 2.0, 'shd': 37, 'nnz': 19}
2 {'fdr': 0.525, 'tpr': 0.475, 'fpr': 4.2, 'shd': 31, 'nnz': 40}
3 {'fdr': 0.5625, 'tpr': 0.35, 'fpr': 3.6, 'shd': 32, 'nnz': 32}
4 {'fdr': 0.59375, 'tpr': 0.325, 'fpr': 3.8, 'shd': 38, 'nnz': 32}


FDR: 0.5725 ± 0.0485
SHD: 34.8000 ± 2.7857
TPR: 0.3250 ± 0.0880
NNZ: 30.4000 ± 6.7705
Test squared loss on original values: 723.6989


std_lambda: 10 and learning_rate: 0.01
0 {'fdr': 0.375, 'tpr': 0.125, 'fpr': 0.6, 'shd': 36, 'nnz': 8}
1 {'fdr': 0.3333333333333333, 'tpr': 0.1, 'fpr': 0.4, 'shd': 37, 'nnz': 6}
2 {'fdr': 0.8888888888888888, 'tpr': 0.025, 'fpr': 1.6, 'shd': 41, 'nnz': 9}
3 {'fdr': 0.5, 'tpr': 0.05, 'fpr': 0.4, 'shd': 39, 'nnz': 4}
4 {'fdr': 0.4166666666666667, 'tpr': 0.175, 'fpr': 1.0, 'shd': 35, 'nnz': 12}


FDR: 0.5028 ± 0.2008
SHD: 37.6000 ± 2.1541
TPR: 0.0950 ± 0.0534
NNZ: 7.80